In [66]:
# Imports
import os
import numpy as np
from astropy.table import Table

# For looking through the table by eye
from IPython.display import display, HTML

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["font.family"] = "serif"
rcParams["font.serif"] = "Times New Roman"
%matplotlib inline
#%matplotlib notebook
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = [r'\usepackage{amsmath} \usepackage{bm} \usepackage{physics}']
%config InlineBackend.figure_format = 'retina' # For high quality figures
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the matched tables

In [67]:
data_folder = "data-2019-09-23-nickel-joey-anne"
OVERWRITE   = True

In [68]:
ngc6819_v_tab = Table.read(os.path.join(data_folder, 'photometry/ngc6819_v_matched_iter_3.dat'), format='ascii')
ngc6819_b_tab = Table.read(os.path.join(data_folder, 'photometry/ngc6819_b_matched_iter_3.dat'), format='ascii')

ngc6819_v_pd = ngc6819_v_tab.to_pandas()
ngc6819_b_pd = ngc6819_b_tab.to_pandas()

## Look for saturated sources (probably in B) and get rid of them

As a proxy, use rows which have bad values for the flux uncertainty estimation.

In [69]:
bad_row_inds = ngc6819_b_pd.index[ngc6819_b_pd['flux_unc'] < 10.].tolist()

### Remove these rows from both dataframes

In [70]:
ngc6819_v_pd_cleaned = ngc6819_v_pd.drop(index=bad_row_inds)
ngc6819_b_pd_cleaned = ngc6819_b_pd.drop(index=bad_row_inds)

print("{} rows dropped for suspiciously low flux uncertainty".format(len(ngc6819_v_pd) - len(ngc6819_v_pd_cleaned))) # Difference is the number of rows dropped

17 rows dropped for suspiciously low flux uncertainty


## Look through the dataframes by eye to check for other outliers that we should get rid of

In [71]:
display(HTML(ngc6819_b_pd_cleaned.to_html()))

,x_0,y_0,flux_0,id,group_id,x_fit,y_fit,flux_fit,flux_unc,x_0_unc,y_0_unc
0,4.595516,448.219348,24948.260926,264,255,3.342851,446.152331,23534.571597,1578.503781,0.194266,0.195070
1,12.228354,528.342185,3193.357479,317,306,11.686333,526.361525,3078.591968,162.410568,0.150914,0.152622
2,14.028523,576.946693,79258.004381,338,326,13.450515,575.064189,73181.372158,3556.615040,0.139102,0.141696
3,17.551507,503.717177,1316.501466,301,291,17.039488,501.263933,1279.532666,99.506627,0.222760,0.236216
4,23.025630,187.295096,516.717936,112,107,22.974483,184.348808,711.405515,77.484032,0.311101,0.329416
5,38.923556,949.258298,109327.389769,473,460,38.282598,947.305932,100545.542201,5105.707273,0.145425,0.147014
6,44.209713,630.000193,1219.370929,365,353,43.017018,628.208690,1200.231325,87.695208,0.209587,0.212118
7,48.975969,64.295448,2735.767708,41,37,48.294401,62.054829,2719.140887,139.614577,0.147026,0.149914
8,52.140761,350.230872,20811.067337,217,211,51.469340,348.176824,19482.235150,970.179568,0.142521,0.144764
9,52.771278,655.098115,15537.241981,377,364,52.030062,653.111533,14506.460566,717.804041,0.141923,0.144035


In [72]:
high_x_0_unc_inds = ngc6819_b_pd_cleaned.index[ngc6819_b_pd_cleaned['x_0_unc'] > 1.].tolist()
ngc6819_v_pd_cleaned_00 = ngc6819_v_pd_cleaned.drop(index=high_x_0_unc_inds)
ngc6819_b_pd_cleaned_00 = ngc6819_b_pd_cleaned.drop(index=high_x_0_unc_inds)

print("{} rows dropped for large pixel uncertainty".format(len(ngc6819_v_pd_cleaned) - len(ngc6819_v_pd_cleaned_00)))

19 rows dropped for large pixel uncertainty


In [73]:
high_flux_unc_inds = ngc6819_b_pd_cleaned_00.index[ngc6819_b_pd_cleaned_00['flux_unc']/ \
                                                   ngc6819_b_pd_cleaned_00['flux_fit'] > 0.1].tolist()
ngc6819_v_pd_cleaned_01 = ngc6819_v_pd_cleaned_00.drop(index=high_flux_unc_inds)
ngc6819_b_pd_cleaned_01 = ngc6819_b_pd_cleaned_00.drop(index=high_flux_unc_inds)

print("{} rows dropped for large flux uncertainty".format(len(ngc6819_v_pd_cleaned_00) - len(ngc6819_v_pd_cleaned_01)))

6 rows dropped for large flux uncertainty


In [74]:
display(HTML(ngc6819_b_pd_cleaned_01.to_html()))

,x_0,y_0,flux_0,id,group_id,x_fit,y_fit,flux_fit,flux_unc,x_0_unc,y_0_unc
0,4.595516,448.219348,24948.260926,264,255,3.342851,446.152331,23534.571597,1578.503781,0.194266,0.195070
1,12.228354,528.342185,3193.357479,317,306,11.686333,526.361525,3078.591968,162.410568,0.150914,0.152622
2,14.028523,576.946693,79258.004381,338,326,13.450515,575.064189,73181.372158,3556.615040,0.139102,0.141696
3,17.551507,503.717177,1316.501466,301,291,17.039488,501.263933,1279.532666,99.506627,0.222760,0.236216
5,38.923556,949.258298,109327.389769,473,460,38.282598,947.305932,100545.542201,5105.707273,0.145425,0.147014
6,44.209713,630.000193,1219.370929,365,353,43.017018,628.208690,1200.231325,87.695208,0.209587,0.212118
7,48.975969,64.295448,2735.767708,41,37,48.294401,62.054829,2719.140887,139.614577,0.147026,0.149914
8,52.140761,350.230872,20811.067337,217,211,51.469340,348.176824,19482.235150,970.179568,0.142521,0.144764
9,52.771278,655.098115,15537.241981,377,364,52.030062,653.111533,14506.460566,717.804041,0.141923,0.144035
10,55.214128,893.056976,5942.835085,459,446,54.456757,890.948037,5719.409889,278.323286,0.139279,0.142343


In [75]:
# Convert back to astropy Tables
ngc6819_v_tab = Table.from_pandas(ngc6819_v_pd_cleaned_01)
ngc6819_b_tab = Table.from_pandas(ngc6819_b_pd_cleaned_01)

In [76]:
# Save the cleaned tables
print('OVERWRITE = {}'.format(OVERWRITE))

OVERWRITE = True


In [77]:
try:
    ngc6819_v_tab.write(os.path.join(data_folder, 'photometry/ngc6819_v_matched_cleaned_iter_3.dat'), format='ascii', 
                        overwrite=OVERWRITE)
    ngc6819_b_tab.write(os.path.join(data_folder, 'photometry/ngc6819_b_matched_cleaned_iter_3.dat'), format='ascii', 
                        overwrite=OVERWRITE)
except:
    print("Warning: Did not overwrite files.")